In [8]:
from remote_vgk import *

import numpy as np
import matplotlib.pyplot as plt 

import time

def load_password(path):

    with open(path) as f:
        secrets = json.load(f)
        shifted_pwd = secrets['password']
    pwd = ''
    for s in shifted_pwd:
        pwd += chr(ord(s) - 1)

    return pwd


try:
    window = gw.getWindowsWithTitle("millenicut:19")[0]
except IndexError:
    raise Exception("Window not found")

pwd = load_password('secrets.json')
sesh = DPO_Session('lwp26', pwd)

# Create the AirfoilApp
App = AirfoilApp(sesh.teaching_client, window, "2.3_incidence_aerofoil")

# IMPORTANT SET THE SAME PARAMETERS AS ON THE APP
App.alpha = 2.3
App.M = 0.75
App.Re = 10


def subsample_cpte_plateau(App, M, Re = 10):

    plateau_data = App.get_plateau_cpte(M, Re)
    try:
        alpha1, _, alpha2, _ = plateau_data
    except TypeError:
        return False
    
    if alpha2 - alpha1 < 0.05:
        print(f"Plateau range small enough: {alpha1} - {alpha2}")
        return
    
    # Subsample the plateau
    alphas = np.linspace(alpha1, alpha2, 5)
    for a in alphas:
        App.run(M, a, Re)


Mop = 0.75
Reop = 10
machs = np.arange(Mop + 0.02, Mop + 0.025, 0.005)
alphas = np.arange(2.5, 3.0, 0.01)

nmachs = machs.shape[0]
nalphas = alphas.shape[0]

print(f"Running {nmachs * nalphas} simulations")
timepersim = 31 * 60 / 450 # 450 runs took 31 minutes
estt = timepersim * nmachs * nalphas
print(f"Estimated time: {estt // 60}:{estt % 60:.2f} minutes")


Running 100 simulations
Estimated time: 6.0:53.33 minutes


In [9]:
start_time = time.time()

for m in machs:
    for a in alphas:
        res = App.run(m, a, Reop)

    dt = time.time() - start_time
    print(f"Finished Mach {m} in {dt // 60}:{dt % 60:.2f}")

    subsample_cpte_plateau(App, m)


Finished Mach 0.77 in 0.0:16.06
Plateau range small enough: 1.3211 - 1.3209
Finished Mach 0.775 in 0.0:32.01


In [10]:
App.classify_buffeting()

TypeError: object of type 'AirfoilApp' has no len()